In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/arcee_lite

Mounted at /content/drive
/content/drive/MyDrive/arcee_lite


In [ ]:
#!pip install -q -U torch=='2.0.0'
!pip install "torch==2.5.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install  --upgrade  "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.23.2" "trl==0.8.6" "peft==0.10.0"


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mg7p7zqh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-mg7p7zqh
  Resolved https://github.com/huggingface/transformers to commit 92c5ca9dd70de3ade2af2eb835c96215cc50e815
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10849200 sha256=cc1bafc4d26e8fd97ce640b524e8e9f3f94143a73150939eaa3f28f3bfe5baa0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ar9hx7kc/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
INFO: pip is looking at multiple versio

In [ ]:
import pandas as pd


data_type='Original'
#data_type='Synthetic'

# Load Data
df = pd.read_excel('LR_Dataset_Original_Sythetic_Final.xlsx')
#df = pd.read_excel('LR_Dataset_Original_Sythetic_Experiment_70.xlsx') # 20% of the training and validation sets


if data_type=='Original':
 df = df[df['Source'] == 'Original'] # Only original

df= df[['Sentence','Category' ,'Classification']]

df['Sentence'] = df['Sentence'].str.capitalize()
df=df.sample(frac=1).reset_index(drop=True)
df['Sentence'] = (df['Sentence']
                  .str.strip()
                  .str.replace(r'\n|\r', ' ', regex=True)
                  .str.replace(r'\s{2,}', ' ', regex=True))
df

<ipython-input-4-878308702a4f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentence'] = df['Sentence'].str.capitalize()


,Sentence,Category,Classification
0,The most competitive algorithms made use of si...,OVERALL,TRAINING
1,We draw inspiration from these benchmarks and ...,EXTENSION,VALIDATION
2,Preliminary evidence suggests that burst scs m...,RESULT,TRAINING
3,The result of the experimental study suggests ...,RESULT,TEST
4,"Moreover, to preserve the ordinal relationship...",DESCRIPTION,TRAINING
...,...,...,...
695,A series of studies by the pew research center...,RESULT,TRAINING
696,The arcs were used by vonnegut to express cano...,DESCRIPTION,TEST
697,Although there is a long tradition of conducti...,RESULT,TEST
698,"Crowdadapt [47], crowdui [48], and xdbrowser [...",DESCRIPTION,TRAINING


In [ ]:
def create_instruction(row):
    sentence = row['Sentence']
    instruction = (
                "You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\n"
                "Categories:\n "
                "1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n "
                "2. RESEARCH GAP: Highlights the need for further research within the topic.\n"
                "3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n"
                "4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “found”, “demonstrated”, and “observed” or phrases like “the findings indicate”.\n"
                "5. LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study that hinders generalizability or reliability in a previous study.\n"
                "6. EXTENSION: Describes how the current study addresses or extends previous studies by stating the overall idea, contrasting ideas or elaborating further ideas. It usually uses the words “we” or “our”.\n"
                "7. OTHER: Any text that does not fit the above categories.\n"
                "Procedure:\n"
                "1. Determine whether the subject of the sentence is a topic or a study.\n"
                "2. Identify the most suitable category based on the content. Do not create new categories. Use the categories given above.\n"
                "3. Provide the category number that best fits the sentence. Just provide the category number without any explanation.\n"

                f"Sentence: {sentence}.\n"
            )

    return instruction

In [ ]:
df['instruction'] = df.apply(create_instruction, axis=1)

df = df.rename(columns={'Category': 'response'})

df

,Sentence,response,Classification,instruction
0,The most competitive algorithms made use of si...,OVERALL,TRAINING,You are a researcher that should assign a clas...
1,We draw inspiration from these benchmarks and ...,EXTENSION,VALIDATION,You are a researcher that should assign a clas...
2,Preliminary evidence suggests that burst scs m...,RESULT,TRAINING,You are a researcher that should assign a clas...
3,The result of the experimental study suggests ...,RESULT,TEST,You are a researcher that should assign a clas...
4,"Moreover, to preserve the ordinal relationship...",DESCRIPTION,TRAINING,You are a researcher that should assign a clas...
...,...,...,...,...
695,A series of studies by the pew research center...,RESULT,TRAINING,You are a researcher that should assign a clas...
696,The arcs were used by vonnegut to express cano...,DESCRIPTION,TEST,You are a researcher that should assign a clas...
697,Although there is a long tradition of conducti...,RESULT,TEST,You are a researcher that should assign a clas...
698,"Crowdadapt [47], crowdui [48], and xdbrowser [...",DESCRIPTION,TRAINING,You are a researcher that should assign a clas...


In [ ]:
test_dataset =df[df['Classification'] == 'TEST']
test_dataset

,Sentence,response,Classification,instruction
3,The result of the experimental study suggests ...,RESULT,TEST,You are a researcher that should assign a clas...
5,While a growing number of studies have examine...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...
6,Although several studies have examined the eff...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...
14,Liu et al. combine self-training based on cont...,RESULT,TEST,You are a researcher that should assign a clas...
19,Civera et al. (2020) distinguish opportunity-o...,RESULT,TEST,You are a researcher that should assign a clas...
...,...,...,...,...
687,"Finally, in the area of ontology modularity, d...",OVERALL,TEST,You are a researcher that should assign a clas...
692,"Specifically, in the third study, they perform...",RESULT,TEST,You are a researcher that should assign a clas...
693,"In this work, we propose a differential testin...",EXTENSION,TEST,You are a researcher that should assign a clas...
696,The arcs were used by vonnegut to express cano...,DESCRIPTION,TEST,You are a researcher that should assign a clas...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline

# Model Setup (Quantization and Tokenizer)

#model_name = "arcee-ai/arcee-lite"
model_name="arcee-ai/SuperNova-Medius"
#model_name="arcee-ai/Arcee-Spark"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, #
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config, # Do not include it beacuse is a tiny model.

)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/45.2k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# Tokenizer Initialization
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_bos_token=True,
    trust_remote_code=True,

)

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

In [ ]:
tokenizer.all_special_tokens

['<|im_end|>',
 '<|endoftext|>',
 '<|im_start|>',
 '<|object_ref_start|>',
 '<|object_ref_end|>',
 '<|box_start|>',
 '<|box_end|>',
 '<|quad_start|>',
 '<|quad_end|>',
 '<|vision_start|>',
 '<|vision_end|>',
 '<|vision_pad|>',
 '<|image_pad|>',
 '<|video_pad|>']

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "SuperNova-Medius-lro-finetune/checkpoint-1225")

In [ ]:
def generate_test_prompt(example):
    """Format prompt for training."""
    text = f"<|im_start|>user\n{example['instruction']}<|im_end|>"
    return text

In [ ]:
test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


<ipython-input-13-ca85691956fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


In [ ]:
test_dataset

,Sentence,response,Classification,instruction,text
3,The result of the experimental study suggests ...,RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
5,While a growing number of studies have examine...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
6,Although several studies have examined the eff...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
14,Liu et al. combine self-training based on cont...,RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
19,Civera et al. (2020) distinguish opportunity-o...,RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
...,...,...,...,...,...
687,"Finally, in the area of ontology modularity, d...",OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
692,"Specifically, in the third study, they perform...",RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
693,"In this work, we propose a differential testin...",EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
696,The arcs were used by vonnegut to express cano...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...


In [ ]:
def extract_text(text):
    # Define both markers to search for
    markers = ["<|im_end|>assistant", "assistant\n"]

    # Loop through markers and check if each is in the text
    for marker in markers:
        marker_position = text.find(marker)

        # If the marker is found, extract text after it
        if marker_position != -1:
            return text[marker_position + len(marker):].strip()  # Remove any leading/trailing whitespace

    # Return None if neither marker is found
    return text

In [ ]:
def clean_text(text):

    # Check for keywords and replace text accordingly
    if "OVERALL" in text:
        return "OVERALL"
    if "RESEARCH GAP" in text:
        return "RESEARCH GAP"
    if "DESCRIPTION" in text:
        return "DESCRIPTION"
    if "RESULT" in text:
        return "RESULT"
    if "LIMITATION" in text:
        return "LIMITATION"
    if "EXTENSION" in text:
        return "EXTENSION"
    elif "OTHER" in text:
        return "OTHER"

    # If neither keyword is found, return the original text
    return text

In [ ]:
prompt=test_dataset.iloc[0]['text']
prompt


'<|im_start|>user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “sh

In [ ]:
inputs = tokenizer(prompt,
                return_tensors="pt"
            ).to("cuda")

outputs = ft_model.generate(**inputs, max_new_tokens=15, use_cache=True)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “foun

In [ ]:
clean_text(extract_text(answer))

'OVERALL'

In [ ]:
#pipe = pipeline(task="text-generation",
#                        model=ft_model,
#                        tokenizer=tokenizer,
#                        max_new_tokens = 100,
#                        temperature = 0.0,
#                       )
#result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
#answer = result[0]['generated_text']

In [ ]:
def get_classification_finetuning(data_point,model,tokenizer):
    """
    Gets the classification for a data point using the fine-tuned model.
    """
    inputs = tokenizer(data_point['text'],return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=15, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(clean_text(extract_text(answer)))
    data_point['Prediction_Finetune']=answer  # Assign the result to the data point
    data_point['Prediction_Finetune_Cleaned']=clean_text(extract_text(answer))

    return data_point
# Apply the get_classification function to the dataset using map
test_dataset = test_dataset.apply(lambda row: get_classification_finetuning(row, ft_model, tokenizer), axis=1)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OVERALL


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OVERALL


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

RESULT


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OVERALL


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


DESCRIPTION
RESEARCH GAP


In [ ]:
test_dataset


,Sentence,response,Classification,instruction,text,Prediction_Finetune,Prediction_Finetune_Cleaned
3,The result of the experimental study suggests ...,RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,OVERALL
5,While a growing number of studies have examine...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,OVERALL
6,Although several studies have examined the eff...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,OVERALL
14,Liu et al. combine self-training based on cont...,RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,OVERALL
19,Civera et al. (2020) distinguish opportunity-o...,RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,DESCRIPTION
...,...,...,...,...,...,...,...
687,"Finally, in the area of ontology modularity, d...",OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,OVERALL
692,"Specifically, in the third study, they perform...",RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,RESULT
693,"In this work, we propose a differential testin...",EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,OVERALL
696,The arcs were used by vonnegut to express cano...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,user\nYou are a researcher that should assign ...,DESCRIPTION


In [ ]:
#test_dataset.to_csv('Arcee-Spark_FT_Test_LoRA3_Augmented.csv', index=False)
test_dataset.to_csv('SuperNova-Medius_FT_LoRA_DoubleCheck.csv', index=False)
